# Treinando POS-Taggers para sentenças em Português com NLTK

Neste notebook vamos treinar POS-taggers em cima do conjunto de dados [Mac-Morpho](http://nilc.icmc.usp.br/macmorpho/), que é composto de sentenças com as POS taggeadas. Para maiores informações das classificações do corpus, consulte o [Manual](http://nilc.icmc.usp.br/macmorpho/macmorpho-manual.pdf).

Vamos fazer o treinamento de vários POS-taggers utilizando as ferramentas do próprio NLTK. 
Para treinamento e uso dos modelos, certifique-se que tenha baixado os seguintes itens do nltk, via o comando `nltk.download()`:
- Corpus Mac-Morpho
- Punkt tokenizer

In [2]:
import nltk
from nltk import word_tokenize
import numpy as np
import joblib 

In [3]:
dataset = list(nltk.corpus.mac_morpho.tagged_sents())

In [4]:
# exemplo de sentença taggeada
dataset[10000]

[('É', 'V'),
 ('mais', 'ADV'),
 ('provável', 'ADJ'),
 ('que', 'KS'),
 ('essa', 'PROADJ'),
 ('amostra', 'N'),
 ('seja', 'V'),
 ('representativa', 'ADJ'),
 ('de', 'PREP'),
 ('toda', 'PROADJ'),
 ('a', 'ART'),
 ('população', 'N'),
 ('feminina', 'ADJ'),
 ('de', 'PREP|+'),
 ('a', 'ART'),
 ('cidade', 'N')]

## Divisão de conjunto de dados de treino (80%) e teste (20%)

In [5]:
tot = len(dataset)
tot_train_samples = int(np.ceil(tot*.8))

train_data = dataset[:tot_train_samples]
test_data = dataset[tot_train_samples:]

## Affix Tagger e Default Tagger

Os seguintes classificadores foram construídos de maneira aditiva, começando de um classificador baseline, e montando taggers mais complexos, que se referiam aos taggers criados previamente via a variável `backoff`.

O `DefaultTagger` é o tagger baseline, pois ele aplica o mesmo POS para qualquer token. Nesse caso, os tokens são classificados como nomes (N no MacMorpho). Podemos ver que chutando N para o conjunto de teste acerta cerca de 20% dos tokens.

Já o `AffixTagger` se baseia em sufixos ou prefixos para a classificação. Podemos passar afixos de diferentes comprimentos, na variável `affix_length`, onde:
- Valores positivos se referem a prefixos (Ex: igual, **des**igual)
- Valores negativos se referem a sufixos (Ex: japon**ês**, ingl**ês**, portugu**ês**)

Como na língua portuguesa o jeito de como a palavra termina pode indicar a classe da palavra (como em verbos, línguas, gerúndio etc), foi utilizado vários taggers de sufixos em cadeia, com cada um aumentando ligeiramente a performance de classificação, chegando a 36.7%.

In [9]:
t_def = nltk.DefaultTagger('N')
t_affix2 = nltk.AffixTagger(train_data, affix_length=-2, backoff=t_def)
t_affix3 = nltk.AffixTagger(train_data, affix_length=-3, backoff=t_affix2)
t_affix4 = nltk.AffixTagger(train_data, affix_length=-4, backoff=t_affix3)
t_affix5 = nltk.AffixTagger(train_data, affix_length=-5, backoff=t_affix4)
t_affix6 = nltk.AffixTagger(train_data, affix_length=-6, backoff=t_affix5)

acc_def = t_def.evaluate(test_data) * 100
acc_af2 = t_affix2.evaluate(test_data) * 100
acc_af3 = t_affix3.evaluate(test_data) * 100
acc_af4 = t_affix4.evaluate(test_data) * 100
acc_af5 = t_affix5.evaluate(test_data) * 100
acc_af6 = t_affix6.evaluate(test_data) * 100

print('''Performance dos taggers:
         - Default:                     {:.2f}%
         - Sufixo tamanho 2 + Default:  {:.2f}%
         - Sufixo tamanho 3 + Sufixo 2: {:.2f}%
         - Sufixo tamanho 4 + Sufixo 3: {:.2f}%
         - Sufixo tamanho 5 + Sufixo 4: {:.2f}%
         - Sufixo tamanho 6 + Sufixo 5: {:.2f}%'''.format(acc_def, acc_af2, acc_af3,
                                                          acc_af4, acc_af5, acc_af6))

Performance dos taggers:
         - Default:                     19.68%
         - Sufixo tamanho 2 + Default:  27.29%
         - Sufixo tamanho 3 + Sufixo 2: 32.23%
         - Sufixo tamanho 4 + Sufixo 3: 34.66%
         - Sufixo tamanho 5 + Sufixo 4: 36.24%
         - Sufixo tamanho 6 + Sufixo 5: 36.71%


## Unigram Tagger

Um unigrama se refere a um token, um conjunto de uma palavra. Assim o `UnigramTagger` é o tagger que faz a classificação tendo como contexto apenas uma palavra. O tagger gerado tem um grande salto de performance, chegando a 83.7%

In [10]:
t_uni = nltk.UnigramTagger(train_data, backoff=t_affix5)

acc_uni = t_uni.evaluate(test_data) * 100

print('''Performance dos taggers:
         - Default:                     {:.2f}%
         - Sufixo tamanho 2 + Default:  {:.2f}%
         - Sufixo tamanho 3 + Sufixo 2: {:.2f}%
         - Sufixo tamanho 4 + Sufixo 3: {:.2f}%
         - Sufixo tamanho 5 + Sufixo 4: {:.2f}%
         - Sufixo tamanho 6 + Sufixo 5: {:.2f}%
         - Unigrama + Sufixo 6:         {:.2f}%'''.format(acc_def, acc_af2, acc_af3,
                                                          acc_af4, acc_af5, acc_af6,
                                                          acc_uni))

Performance dos taggers:
         - Default:                     19.68%
         - Sufixo tamanho 2 + Default:  27.29%
         - Sufixo tamanho 3 + Sufixo 2: 32.23%
         - Sufixo tamanho 4 + Sufixo 3: 34.66%
         - Sufixo tamanho 5 + Sufixo 4: 36.24%
         - Sufixo tamanho 6 + Sufixo 5: 36.71%
         - Unigrama + Sufixo 6:         83.70%


## Bigram e Trigram Tagger

O `BigramTagger` e o `TrigramTagger`, utilizam as tags anteriorer como parte do contexto, útil em casos onde um token pode ter diferentes usos dependendo do contexto. Os classificadores tem uma ligeira melhora, om acurácia acima de 85%.

In [11]:
t_bi = nltk.BigramTagger(train_data, backoff=t_uni)
t_tri = nltk.TrigramTagger(train_data, backoff=t_bi)

acc_bi = t_bi.evaluate(test_data) * 100
acc_tri = t_tri.evaluate(test_data) * 100

print('''Performance dos taggers:
         - Default:                     {:.2f}%
         - Sufixo tamanho 2 + Default:  {:.2f}%
         - Sufixo tamanho 3 + Sufixo 2: {:.2f}%
         - Sufixo tamanho 4 + Sufixo 3: {:.2f}%
         - Sufixo tamanho 5 + Sufixo 4: {:.2f}%
         - Sufixo tamanho 6 + Sufixo 5: {:.2f}%
         - Unigrama + Sufixo 6:         {:.2f}%
         - Bigrama + Unigrama:          {:.2f}%
         - Trigrama + Bigrama:          {:.2f}%'''.format(acc_def, acc_af2, acc_af3,
                                                          acc_af4, acc_af5, acc_af6,
                                                          acc_uni, acc_bi, acc_tri))

Performance dos taggers:
         - Default:                     19.68%
         - Sufixo tamanho 2 + Default:  27.29%
         - Sufixo tamanho 3 + Sufixo 2: 32.23%
         - Sufixo tamanho 4 + Sufixo 3: 34.66%
         - Sufixo tamanho 5 + Sufixo 4: 36.24%
         - Sufixo tamanho 6 + Sufixo 5: 36.71%
         - Unigrama + Sufixo 6:         83.70%
         - Bigrama + Unigrama:          85.18%
         - Trigrama + Bigrama:          85.19%


## Brill Tagger

O `BrillTagger` é um tagger que se baseia na criação de regras para melhorar o desempenho de classificação. Ele é um tagger que demora um pouco para treinar, mas produz um salto na acurácia, chegando a 92.19%.

In [13]:
templates = nltk.brill.fntbl37()
brill_tagger = nltk.BrillTaggerTrainer(t_tri, templates, trace=True)
brill_tagger = brill_tagger.train(train_data)

acc_brill = brill_tagger.evaluate(test_data) * 100

print('''Performance dos taggers:
         - Default:                     {:.2f}%
         - Sufixo tamanho 2 + Default:  {:.2f}%
         - Sufixo tamanho 3 + Sufixo 2: {:.2f}%
         - Sufixo tamanho 4 + Sufixo 3: {:.2f}%
         - Sufixo tamanho 5 + Sufixo 4: {:.2f}%
         - Sufixo tamanho 6 + Sufixo 5: {:.2f}%
         - Unigrama + Sufixo 6:         {:.2f}%
         - Bigrama + Unigrama:          {:.2f}%
         - Trigrama + Bigrama:          {:.2f}%
         - Brill Tagger + Trigrama:     {:.2f}%'''.format(acc_def, acc_af2, acc_af3,
                                                          acc_af4, acc_af5, acc_af6,
                                                          acc_uni, acc_bi, acc_tri,
                                                          acc_brill))

TBL train (fast) (seqs: 41118; tokens: 913108; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 848255 useful rules.
Selecting rules...
Performance dos taggers:
         - Default:                     19.68%
         - Sufixo tamanho 2 + Default:  27.29%
         - Sufixo tamanho 3 + Sufixo 2: 32.23%
         - Sufixo tamanho 4 + Sufixo 3: 34.66%
         - Sufixo tamanho 5 + Sufixo 4: 36.24%
         - Sufixo tamanho 6 + Sufixo 5: 36.71%
         - Unigrama + Sufixo 6:         83.70%
         - Bigrama + Unigrama:          85.18%
         - Trigrama + Bigrama:          85.19%
         - Brill Tagger + Trigrama:     92.19%


## Classifier-based tagging

O `ClassifierBasedPOSTagger` é um outro tipo de tagger, desta vez não aditivo como os anteriores, mas sim um modelo baseado em um classificador, como o nome diz. O classificador default do NLTK é uma implementação do algoritmo `Naive-Bayes`.

In [30]:
from nltk.tag.sequential import ClassifierBasedPOSTagger

naive_tagger = ClassifierBasedPOSTagger(train=train_data)

acc_naive = naive_tagger.evaluate(test_data) * 100

print('''Performance do tagger:
         - ClassifierBased (Naive Bayes): {:.2f}%'''.format(acc_naive))

Performance do tagger:
         - ClassifierBased (Naive Bayes): 83.97%


## Salvando os taggers

Com os taggers treinados, podemos salvar eles em arquivos `pickle` para uso posterior:

In [60]:
folder = 'trained_POS_taggers/'
joblib.dump(t_affix6,folder+'POS_tagger_affix6.pkl')
joblib.dump(t_uni,folder+'POS_tagger_unigram.pkl')
joblib.dump(t_bi,folder+'POS_tagger_bigram.pkl')
joblib.dump(t_tri,folder+'POS_tagger_trigram.pkl')
joblib.dump(brill_tagger,folder+'POS_tagger_brill.pkl')
joblib.dump(naive_tagger,folder+'POS_tagger_naive.pkl')

['trained_POS_taggers/POS_tagger_naive.pkl']

## Carregando arquivo pkl para uso dos POS-taggers treinados

Para utilizar eles, basta carregar o arquivo `pickle` gerado, como por exemplo, usando a função `load` da biblioteca `joblib`.

In [62]:
teste_tagger = joblib.load(folder+'POS_tagger_brill.pkl')

phrase = 'O rato roeu a roupa do rei de Roma'

teste_tagger.tag(word_tokenize(phrase))

[('O', 'ART'),
 ('rato', 'N'),
 ('roeu', 'V'),
 ('a', 'ART'),
 ('roupa', 'N'),
 ('do', 'KS'),
 ('rei', 'N'),
 ('de', 'PREP'),
 ('Roma', 'NPROP')]

# Comparação dos taggers - Tempo 

Para uma comparação do desempenho dos taggers treinados, vamos medir o tempo gasto para classificar as primeiras 100 sentenças do Mac-Morpho, que tem ao todo 2260 palavras. Para efeitos de comparação, esse teste foi feito em Python 3.6, em uma máquina com processador Intel i7 e 16 GB de RAM.

In [47]:
sent_test = nltk.corpus.mac_morpho.sents()[:100]
tot_words = sum([len(s) for s in list(sent_test)])

print('Total de palavras nas 100 primeiras sentenças do Mac-Morpho: {}'.format(tot_words))

Total de palavras nas 100 primeiras sentenças do Mac-Morpho: 2260


In [37]:
%timeit t_affix6.tag_sents(sent_test)

31.2 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%timeit t_uni.tag_sents(sent_test)

27.5 ms ± 831 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%timeit t_bi.tag_sents(sent_test)

33.6 ms ± 4.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%timeit t_tri.tag_sents(sent_test)

36.6 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%timeit brill_tagger.tag_sents(sent_test)

74.9 ms ± 8.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%timeit naive_tagger.tag_sents(sent_test)

2.87 s ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
words_sec_affix6 = 1/((31.2* 10**-3)/tot_words)
words_sec_uni = 1/((27.5* 10**-3)/tot_words)
words_sec_bi = 1/((33.6* 10**-3)/tot_words)
words_sec_tri = 1/((36.6* 10**-3)/tot_words)
words_sec_brill = 1/((74.9* 10**-3)/tot_words)
words_sec_naive = 1/(2.87/tot_words)

print('''Palavras processadas por segundo (palavras/s):
         - Sufixo tamanho 6 + Sufixo 5:   {:.2f} palavras/s
         - Unigrama + Sufixo 6:           {:.2f} palavras/s
         - Bigrama + Unigrama:            {:.2f} palavras/s
         - Trigrama + Bigrama:            {:.2f} palavras/s
         - Brill Tagger + Trigrama:       {:.2f} palavras/s
         - ClassifierBased (Naive Bayes):   {:.2f} palavras/s'''.format(words_sec_affix6, words_sec_uni,
                                                                        words_sec_bi, words_sec_tri,
                                                                        words_sec_brill, words_sec_naive))

Palavras processadas por segundo (palavras/s):
         - Sufixo tamanho 6 + Sufixo 5:   72435.90 palavras/s
         - Unigrama + Sufixo 6:           82181.82 palavras/s
         - Bigrama + Unigrama:            67261.90 palavras/s
         - Trigrama + Bigrama:            61748.63 palavras/s
         - Brill Tagger + Trigrama:       30173.56 palavras/s
         - ClassifierBased (Naive Bayes):   787.46 palavras/s


# Conclusão

Podemos montar uma tabela final com os dados dos modelos treinados. Podemos ver que o BrillTagger teve a melhor acurácia, sendo um pouco mais lento que os modelos feitos antes dele. E o modelo baseado no Naive-Bayes além de não ter a melhor acurácia, produziu o maior pickle, e foi de longe o mais lento, não indicado para uso para corpus grandes:


| Tagger                 | Acurácia | Palavras/s | Tamanho  |
|------------------------|----------|------------|----------|
| POS_tagger_affix6.pkl  | 36.71%   | 72k        | 386 kB   |
| POS_tagger_unigram.pkl | 83.70%   | **82k**        | 790 kB   |
| POS_tagger_bigram.pkl  | 85.18%   | 67k        | 1.37 MB  |
| POS_tagger_trigram.pkl | 85.19%   | 61k        | 2.05 MB  |
| POS_tagger_brill.pkl   | **92.19%**   | 30k        | 2.09 MB  |
| POS_tagger_naive.pkl   | 83.97%   | 787        | 22.43 MB |